# Location Designator
This example will show you waht location designator are, how to use them and what they are capable of. 

Location Deisgnator are used to semantically describe locations in the world. You could, for example, create a location designator that describes every position where a robot can be placed without colliding with the environment. Location designator can describe locations for:

   * Visibility 
   * Reachability
   * Occupany 
   * URDF Links (for example a table)

To find locations that fit the given constrains, location designator create Costmaps. Costmaps are a 2D distribution that have a value greater than 0 for every position that fits the costmap criteria.

Location designator work similar to other designator, meaning you have to create a location designator description which describes the location. This description can then be resolved to the actual 6D pose on runtime.

## Occupancy

We will start with a simple location designator that describes a location where the robot can be placed without colliding with the environment. To do this we need a BulletWorld since the costmaps are mostly created from the current state of the BulletWorld. 

In [1]:
from pycram.bullet_world import BulletWorld, Object
from pycram.pose import Pose

world = BulletWorld()
kitchen = Object("kitchen", "environment", "kitchen.urdf")

Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Scalar element defined multiple times: limit
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Scalar element defined multiple times: limit


In [12]:
world.exit()

Next up we will create the location designator description, the ```CostmapLocation``` that we will be using needs a target as parameter. This target describes what the location designator is for, this could either be a pose or object that the robot should be able to see or reach.

In this case we only want poses where the robot can be placed, this is the default behaviour of the location designator which we will be extending later. 

In [3]:
from pycram.designators.location_designator import CostmapLocation

target = kitchen.get_pose()

location_description = CostmapLocation(target)

pose = location_description.resolve()

print(pose)

CostmapLocation.Location(pose=header: 
  seq: 0
  stamp: 
    secs: 1690274742
    nsecs: 490613698
  frame_id: "map"
pose: 
  position: 
    x: 0.32
    y: 0.46
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.8863025691598214
    w: -0.46310663556107684, reachable_arms=None)


## Reachable
Next we want to locations from where the robot can reach a specific point, like an object the robot should pick up. This can also be done with the ```CostmapLocation``` description, but this time we need to provide an additional argument. The additional argument is the robo which should be able to reach the pose. 

Since a robot is needed we will use the PR2 and use a milk as a target point for the robot to reach. The torso of the PR2 will be set to 0.2 since otherwise the arms of the robot will be too low to reach on the countertop.


In [4]:
pr2 = Object("pr2", "robot", "pr2.urdf")
pr2.set_joint_state("torso_lift_joint", 0.2)
milk = Object("milk", "milk", "milk.stl", pose=Pose([1.3, 1, 0.9]))


Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


In [5]:
from pycram.designators.location_designator import CostmapLocation
from pycram.designators.object_designator import BelieveObject

target = BelieveObject(names=["milk"]).resolve()
robot_desig = BelieveObject(names=["pr2"]).resolve()

location_description = CostmapLocation(target=target, reachable_for=robot_desig)

print(location_description.resolve())

CostmapLocation.Location(pose=header: 
  seq: 0
  stamp: 
    secs: 1690274760
    nsecs:  13957262
  frame_id: "map"
pose: 
  position: 
    x: 0.52
    y: 0.94
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.038376519503587225
    w: 0.99926335004882, reachable_arms=['left'])


As you can see we get a pose near the countertop where the robot can be placed without colliding with it. Furthermore, we get a list of arms with which the robot can reach the given object.

## Visibile
The ```CostmapLocation``` can also find position from which the robot can see a given object or location. This is very similar to how rechable locations are described, meaning we provide a object designator or a pose and a robot designator but this time we use the ```visible_for``` parameter.  

For this example we need the milk as well as the PR2, so if you did not spawn them during the previous location designator you can spawn them with the following cell.  

In [ ]:
pr2 = Object("pr2", "robot", "pr2.urdf")
milk = Object("milk", "milk", "milk.stl", pose=Pose([1.3, 1, 0.9]))

In [6]:
from pycram.designators.location_designator import CostmapLocation
from pycram.designators.object_designator import BelieveObject

target = BelieveObject(names=["milk"]).resolve()
robot_desig = BelieveObject(names=["pr2"]).resolve()

location_description = CostmapLocation(target=target, visible_for=robot_desig)

print(location_description.resolve())

CostmapLocation.Location(pose=header: 
  seq: 0
  stamp: 
    secs: 1690274771
    nsecs: 450996160
  frame_id: "map"
pose: 
  position: 
    x: 0.7000000000000001
    y: 0.040000000000000036
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.4847685323929452
    w: 0.8746424812468178, reachable_arms=None)


## Semantic 
Semantic location designator are used to create location descriptions for semantic entities, like a table. An example of this is: You have a robot that picked up an object and should place it on a table. Semantic location designator then allows to find poses that are on this table.

Semantic location designator need an object from which the target entity is a part and the urdf link representing the entity. In this case we want a position on the kitchen island, so we have to provide the kitchen object designator since the island is a part of the kitchen and the link name of the island surface. 

For this example we need the kitchen as well as the milk. If you spawned them in one of the previous examples you don't need to execute the following cell.

In [8]:
kitchen = Object("kitchen", "environment", "kitchen.urdf")
milk = Object("milk", "milk", "milk.stl")

In [7]:
from pycram.designators.location_designator import SemanticCostmapLocation 
from pycram.designators.object_designator import BelieveObject

kitchen_desig = BelieveObject(names=["kitchen"]).resolve()
milk_desig = BelieveObject(names=["milk"]).resolve()

location_description = SemanticCostmapLocation(urdf_link_name="kitchen_island_surface", 
                                               part_of=kitchen_desig,
                                              for_object=milk_desig)

print(location_description.resolve())

SemanticCostmapLocation.Location(pose=header: 
  seq: 0
  stamp: 
    secs: 1690274773
    nsecs: 737708330
  frame_id: "map"
pose: 
  position: 
    x: -1.2074999952316285
    y: 1.019200015068054
    z: 0.9398907270729542
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.6339889056055381
    w: 0.7733421413379024)


## Location Designator as Generator
Location designator descriptions implement an iter method, so they can be used as generators which generate valid poses for the location described in the description. This can be useful if the first pose does not work for some reason. 

We will see this at the example of a location designator for visibility. For this example we need the milk, if you already have a milk spawned in you world you can ignore the following cell.

In [ ]:
milk = Object("milk", "milk", "milk.stl")

In [8]:
from pycram.designators.location_designator import CostmapLocation
from pycram.designators.object_designator import BelieveObject

target = BelieveObject(names=["milk"]).resolve()
robot_desig = BelieveObject(names=["pr2"]).resolve()

location_description = CostmapLocation(target=target, visible_for=robot_desig)

for pose in location_description:
    print(pose.pose)

header: 
  seq: 0
  stamp: 
    secs: 1690274779
    nsecs: 531814098
  frame_id: "map"
pose: 
  position: 
    x: 0.7000000000000001
    y: 0.040000000000000036
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.4847685323929452
    w: 0.8746424812468178
header: 
  seq: 0
  stamp: 
    secs: 1690274779
    nsecs: 552193641
  frame_id: "map"
pose: 
  position: 
    x: 0.7000000000000001
    y: 0.14
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.462503191327324
    w: 0.8866176165698721
header: 
  seq: 0
  stamp: 
    secs: 1690274779
    nsecs: 563171625
  frame_id: "map"
pose: 
  position: 
    x: 0.7000000000000001
    y: 0.07999999999999996
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.47630463962315017
    w: 0.8792803251941107
header: 
  seq: 0
  stamp: 
    secs: 1690274779
    nsecs: 573374509
  frame_id: "map"
pose: 
  position: 
    x: 0.7000000000000001
    y: 0.6
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.2897841486884302
    w: 0

    w: -0.9762640882454053
header: 
  seq: 0
  stamp: 
    secs: 1690274779
    nsecs: 974899768
  frame_id: "map"
pose: 
  position: 
    x: 0.7000000000000001
    y: 1.3
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.22975292054736132
    w: -0.9732489894677301
header: 
  seq: 0
  stamp: 
    secs: 1690274779
    nsecs: 979987144
  frame_id: "map"
pose: 
  position: 
    x: 0.7000000000000001
    y: 1.32
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.24253562503633316
    w: -0.9701425001453319
header: 
  seq: 0
  stamp: 
    secs: 1690274779
    nsecs: 985614061
  frame_id: "map"
pose: 
  position: 
    x: 0.7000000000000001
    y: 1.34
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.2549292197369575
    w: -0.9669597162882775
header: 
  seq: 0
  stamp: 
    secs: 1690274779
    nsecs: 991364717
  frame_id: "map"
pose: 
  position: 
    x: 0.7000000000000001
    y: 1.3599999999999999
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.266933

header: 
  seq: 0
  stamp: 
    secs: 1690274780
    nsecs: 388308048
  frame_id: "map"
pose: 
  position: 
    x: 0.68
    y: 0.92
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.06411760041507647
    w: 0.9979423496961197
header: 
  seq: 0
  stamp: 
    secs: 1690274780
    nsecs: 393875360
  frame_id: "map"
pose: 
  position: 
    x: 0.8
    y: 1.02
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.019988012385059234
    w: -0.9998002197243681
header: 
  seq: 0
  stamp: 
    secs: 1690274780
    nsecs: 398878574
  frame_id: "map"
pose: 
  position: 
    x: 0.68
    y: 0.9
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.07987139440172826
    w: 0.9968051767302994
header: 
  seq: 0
  stamp: 
    secs: 1690274780
    nsecs: 403767585
  frame_id: "map"
pose: 
  position: 
    x: 0.68
    y: 0.88
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.09544671133600839
    w: 0.9954345409393531
header: 
  seq: 0
  stamp: 
    secs: 1690274780
    nsecs: 409

    w: -0.9909572690600927
header: 
  seq: 0
  stamp: 
    secs: 1690274780
    nsecs: 602478742
  frame_id: "map"
pose: 
  position: 
    x: 0.7200000000000001
    y: 1.1400000000000001
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.1181477706660459
    w: -0.9929960243055576
header: 
  seq: 0
  stamp: 
    secs: 1690274780
    nsecs: 607594251
  frame_id: "map"
pose: 
  position: 
    x: 0.7200000000000001
    y: 1.12
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.1018321670517163
    w: -0.9948015931599383
header: 
  seq: 0
  stamp: 
    secs: 1690274780
    nsecs: 612650632
  frame_id: "map"
pose: 
  position: 
    x: 0.7200000000000001
    y: 1.1
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.08526395561915981
    w: -0.9963583983046331
header: 
  seq: 0
  stamp: 
    secs: 1690274780
    nsecs: 617755174
  frame_id: "map"
pose: 
  position: 
    x: 0.7200000000000001
    y: 1.5
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.34827434

    w: 0.9995685001977093
header: 
  seq: 0
  stamp: 
    secs: 1690274781
    nsecs:  40457248
  frame_id: "map"
pose: 
  position: 
    x: 0.28
    y: 0.9
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.04884401130138718
    w: 0.9988064189621481
header: 
  seq: 0
  stamp: 
    secs: 1690274781
    nsecs:  50108432
  frame_id: "map"
pose: 
  position: 
    x: 0.28
    y: 0.86
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.0681484423724803
    w: 0.9976751925362306
header: 
  seq: 0
  stamp: 
    secs: 1690274781
    nsecs:  55038452
  frame_id: "map"
pose: 
  position: 
    x: 0.28
    y: 0.84
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.07771896040148094
    w: 0.9969753072138312
header: 
  seq: 0
  stamp: 
    secs: 1690274781
    nsecs:  60004949
  frame_id: "map"
pose: 
  position: 
    x: 0.28
    y: 0.8200000000000001
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.08722511445123822
    w: 0.9961886264202018
header: 
  seq: 0
  stamp:

header: 
  seq: 0
  stamp: 
    secs: 1690274781
    nsecs: 254323244
  frame_id: "map"
pose: 
  position: 
    x: 0.4600000000000001
    y: 0.31999999999999995
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.333732774306416
    w: 0.9426677226646422
header: 
  seq: 0
  stamp: 
    secs: 1690274781
    nsecs: 259426355
  frame_id: "map"
pose: 
  position: 
    x: 0.4600000000000001
    y: 0.33999999999999997
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.3268651565841618
    w: 0.945070986440284
header: 
  seq: 0
  stamp: 
    secs: 1690274781
    nsecs: 264454603
  frame_id: "map"
pose: 
  position: 
    x: 0.4600000000000001
    y: 0.36
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.3198189174888669
    w: 0.9474786857846721
header: 
  seq: 0
  stamp: 
    secs: 1690274781
    nsecs: 269366741
  frame_id: "map"
pose: 
  position: 
    x: 0.4600000000000001
    y: 0.38
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.31259072316227876
    w: 0.

    w: -0.8944271909999159
header: 
  seq: 0
  stamp: 
    secs: 1690274781
    nsecs: 492344141
  frame_id: "map"
pose: 
  position: 
    x: 0.68
    y: 0.48
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.3419126035072468
    w: 0.9397317551104125
header: 
  seq: 0
  stamp: 
    secs: 1690274781
    nsecs: 497493743
  frame_id: "map"
pose: 
  position: 
    x: 0.4600000000000001
    y: 1.5
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.26524245096731186
    w: -0.9641817474962141
header: 
  seq: 0
  stamp: 
    secs: 1690274781
    nsecs: 502514123
  frame_id: "map"
pose: 
  position: 
    x: 0.4600000000000001
    y: 1.52
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.2736191269923746
    w: -0.9618381222138842
header: 
  seq: 0
  stamp: 
    secs: 1690274781
    nsecs: 507708787
  frame_id: "map"
pose: 
  position: 
    x: 0.4600000000000001
    y: 1.54
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.281799152432585
    w: -0.95947341687

header: 
  seq: 0
  stamp: 
    secs: 1690274781
    nsecs: 706691741
  frame_id: "map"
pose: 
  position: 
    x: 0.6
    y: 0.92
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.056865311674147184
    w: 0.9983818589739109
header: 
  seq: 0
  stamp: 
    secs: 1690274781
    nsecs: 711645841
  frame_id: "map"
pose: 
  position: 
    x: 0.6
    y: 0.9
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.07088902009067935
    w: 0.9974842088126424
header: 
  seq: 0
  stamp: 
    secs: 1690274781
    nsecs: 716599464
  frame_id: "map"
pose: 
  position: 
    x: 0.6
    y: 0.88
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.08478721951666214
    w: 0.9963990803923062
header: 
  seq: 0
  stamp: 
    secs: 1690274781
    nsecs: 721657037
  frame_id: "map"
pose: 
  position: 
    x: 0.6
    y: 0.86
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.09853761796664222
    w: 0.9951333266680702
header: 
  seq: 0
  stamp: 
    secs: 1690274781
    nsecs: 72700762

header: 
  seq: 0
  stamp: 
    secs: 1690274782
    nsecs: 129829168
  frame_id: "map"
pose: 
  position: 
    x: 0.74
    y: 0.76
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.20106587226819733
    w: 0.9795777228015289
header: 
  seq: 0
  stamp: 
    secs: 1690274782
    nsecs: 134856700
  frame_id: "map"
pose: 
  position: 
    x: 0.74
    y: 0.78
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.18607593417695478
    w: 0.9825353666510807
header: 
  seq: 0
  stamp: 
    secs: 1690274782
    nsecs: 140148878
  frame_id: "map"
pose: 
  position: 
    x: 0.74
    y: 0.8
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.1706723299266206
    w: 0.9853278417853718
header: 
  seq: 0
  stamp: 
    secs: 1690274782
    nsecs: 145434379
  frame_id: "map"
pose: 
  position: 
    x: 0.74
    y: 0.8200000000000001
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.15487323641406808
    w: 0.9879343503708291
header: 
  seq: 0
  stamp: 
    secs: 1690274782
   

header: 
  seq: 0
  stamp: 
    secs: 1690274782
    nsecs: 348346710
  frame_id: "map"
pose: 
  position: 
    x: 0.26
    y: 0.8200000000000001
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.08558473621082238
    w: 0.9963308952992093
header: 
  seq: 0
  stamp: 
    secs: 1690274782
    nsecs: 353662014
  frame_id: "map"
pose: 
  position: 
    x: 0.26
    y: 0.8
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.09485133899780393
    w: 0.9954914482256106
header: 
  seq: 0
  stamp: 
    secs: 1690274782
    nsecs: 359004020
  frame_id: "map"
pose: 
  position: 
    x: 0.26
    y: 0.78
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.10404397359063816
    w: 0.994572697976106
header: 
  seq: 0
  stamp: 
    secs: 1690274782
    nsecs: 364920377
  frame_id: "map"
pose: 
  position: 
    x: 0.26
    y: 0.76
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.11315653826752592
    w: 0.9935771725675414
header: 
  seq: 0
  stamp: 
    secs: 1690274782
   

## Accessing Locations
Accessing describes a location from which the robot can open a drawer. The drawer is specified by a ObjetcPart designator which describes the handle of the drawer.

At the moment this location designator only works in the apartment environment, so please remove the kitchen if you spawned it in a previous example. Furthermore, we need a robot so we also spawn the pr2 if it isn't spawned already.

In [9]:
kitchen.remove()

In [10]:
apartment = Object("apartment", "environment", "apartment.urdf")

Unknown tag "material" in /robot[@name='apartment']/link[@name='coffe_machine']/collision[1]
Unknown tag "material" in /robot[@name='apartment']/link[@name='coffe_machine']/collision[1]


In [10]:
pr2 = Object("pr2", "robot", "pr2.urdf")
pr2.set_joint_state("torso_lift_joint", 0.25)

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


In [11]:
from pycram.designators.object_designator import *
from pycram.designators.location_designator import *

apartment_desig = BelieveObject(names=["apartment"])
handle_desig = ObjectPart(names=["handle_cab10_t"], part_of=apartment_desig.resolve())
robot_desig = BelieveObject(names=["pr2"])

access_location = AccessingLocation(handle_desig.resolve(), robot_desig.resolve()).resolve()
print(access_location.pose)

header: 
  seq: 0
  stamp: 
    secs: 1690274807
    nsecs: 896183252
  frame_id: "map"
pose: 
  position: 
    x: 1.8074915790557862
    y: 2.7473597526550293
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.5893608715092853
    w: -0.8078698924541103
